In [2]:
import random
from typing import List, Tuple
from typing_extensions import Self

In [3]:
class DecisionVariable:
    def __init__(self, shape: List[int], default_value=1):
        if len(shape) == 0 or 0 in shape:
            raise Exception("Dimensão inválida informada")
        
        self.shape = shape
        self.default_value = default_value
        self.values = self.iterate(0)

    def iterate(self, index: int):
        return [self.initialize(index) for i in range(0, self.shape[index])]

    def initialize(self, index: int):
        if len(self.shape) > index + 1:
            return self.iterate(index + 1)
        else:
            return self.default_value
        
    def crossover(self, other: Self, axis: int) -> Tuple[Self, Self]:
        if self.shape != other.shape:
            raise Exception("As variáveis precisam ter a mesma dimensão")
        
        if axis >= len(self.shape):
            raise Exception("O eixo solicitado não existe na dimensão")
        
        if self.shape[axis] < 2:
            raise Exception("A dimensão é pequena demais para o split")
        
        # Define o local de corte
        split = random.randint(1, self.shape[axis] - 1)
        
        # Define a função recursiva de cópia
        def copy(
            one: list | float,
            two: list | float,
            current_index: List[int],
            deep=0
        ):
            if(type(one) != list):
                # Caso trivial, percorrendo os valores e atualizando o índice
                value = one if current_index[axis] < split else two
                current_index[deep] = current_index[deep] + 1
                return value
            
            else:
                # Atualiza o índice de profundidade
                deep = deep + 1
                def update_i(list_i: List[int], i: int):
                    return list_i[:deep] + [i] + (len(list_i[deep:]) - 1) * [0]

                # Aciona a cópia dentro da nova lista
                return [
                    copy(one[i], two[i], update_i(current_index, i), deep=deep)
                    for i in range(0, len(one))
                ]
        
        # Define a estrutura inicial de cópia e inicialização do processo
        base_index = [0 for _ in range(1, len(self.shape))]
        data1 = [
            copy(self.values[i], other.values[i], [i] + base_index)
            for i in range(0, self.shape[0])
        ]
        data2 = [
            copy(other.values[i], self.values[i], [i] + base_index)
            for i in range(0, self.shape[0])
        ]

        # Cria instâncias de variáveis de decisão utilizando os dados criados
        dv1 = DecisionVariable(self.shape)
        dv1.values = data1

        dv2 = DecisionVariable(self.shape)
        dv2.values = data2

        return dv1, dv2
        

dv1 = DecisionVariable([4, 3, 2])
print("Variável de decisão 1")
print(dv1.values)
print()

dv2 = DecisionVariable([4, 3, 2], default_value=3)
print("Variável de decisão 2")
print(dv2.values)
print()

cv1, cv2 = dv1.crossover(dv2, 0)
print("Crosover 1")
print(cv1.values)
print()
print("Crossover 2")
print(cv2.values)
print()


Variável de decisão 1
[[[1, 1], [1, 1], [1, 1]], [[1, 1], [1, 1], [1, 1]], [[1, 1], [1, 1], [1, 1]], [[1, 1], [1, 1], [1, 1]]]

Variável de decisão 2
[[[3, 3], [3, 3], [3, 3]], [[3, 3], [3, 3], [3, 3]], [[3, 3], [3, 3], [3, 3]], [[3, 3], [3, 3], [3, 3]]]

Crosover 1
[[[1, 1], [1, 1], [1, 1]], [[1, 1], [1, 1], [1, 1]], [[1, 1], [1, 1], [1, 1]], [[3, 3], [3, 3], [3, 3]]]

Crossover 2
[[[3, 3], [3, 3], [3, 3]], [[3, 3], [3, 3], [3, 3]], [[3, 3], [3, 3], [3, 3]], [[1, 1], [1, 1], [1, 1]]]



In [ ]:
class Solution:
    def __init__(
        self, times: int, batchs: int, crops: int, animals: int,
        crop_time: List[int], animal_area: List[float], batch_area: float,
        animal_growth_time: List[int], animal_manure: List[float]
    ):
        self.times = times
        self.crops = crops
        self.batchs = batchs
        self.animals = animals
        self.crop_time = crop_time
        self.batch_area = batch_area
        self.animal_area = animal_area
        self.animal_growth_time = animal_growth_time

    def random(self):
        '''
        Estratégia de geração de solução:
        Define n como o total de lotes
        Define m como o total de dias
        Inicializa com 0 uma matriz (A) n x m, que armazena ações sobre ocupação dos lotes
        Inicializa com 0 uma matriz (C) n x m, que armazena compras de animais
        Inicializa com 0 uma matriz (V) n x m, que armazena vendas de animais
        Inicializa com 0 uma matriz (M) n x m, que armazena os abates de animais

        Percorre os lotes disponíveis:
            Percorre os dias disponíveis:
                Define enum de estado atual:
                    Caso seja o dia 0, marca enum com "ação inicial"
                    Caso os dias anteriores estejam ocupados por plantio (C+ID) e o tempo de crescimento acabou, marca com "ação inicial"
                    Caso o dia anterior esteja ocupado com criação de animal (A+ID), marca com enum "criação de animal"
                    Caso o dia anterior esteja ocupado com plantio (C+ID) e o tempo de crescimento não acabou, marca como "plantio"

                Define tempo imodificável (k) como 0:
                    Volta no tempo até chegar no momento em que a ação em (A) se altera
                    Atualiza (k) com a quantidade de passos até acontecer a troca

                Se o enum de estado for "ação inicial", decide se vai deixar vazio (0), ocupar com plantio (C+ID) ou ocupar com animal (A+ID)
                    Se a decisão for não utilizar, marca 0 em (A)
                    Se a decisão for ocupar com plantio (C+ID), seleciona o ID de um cultivo de marca em (A)
                    Se a decisão for ocupar com animal (A+ID), seleciona o ID de um animal e marca em (A)
                        Define o total de lotes de animais possíveis para o lote como (l)
                        Soma o total de registros na matriz de compra (C) desde o dia (k)
                        Soma o total de registros da matriz de venda (V) desde o dia (k)
                        Define o total disponível para compra como (t) = soma(C) - soma(V)

                        De (k) até o dia da iteração, recupera na matriz de compras de animais a quantidade de animais comprados com no mínimo o tempo de crescimento (a1)
                        De (k) Até o dia da iteração, recupera na matriz de vendas de animais a quantidade vendida (a2)
                        Define o estoque disponível para a venda como (a1) - (a2) = (a3)

                        Decide se irá comprar, vender ou manter animais
                        Decisão de compra:
                            Selecione entre 0 e (l)-(t) a quantidade de lotes de animais a serem comprados
                            Marca em (C) a quantidade de animais comprados
                        Decisão de venda, apenas se (a3) > 0:
                            Seleciona entre 0 e (a3) a quantidade de lotes de animais vendidos
                            Marca em (V) a quantidade de animais vendidos
                        Decisão de manter:
                            Não atualiza as matrizes (C) e (V)

                Se o enum de estado for "plantio", marca a matriz (A) com a mesma ação que a anterior

                Se o enum de estado for "criação de animal":
                    Define o período de busca (p) como o tempo de crescimento para venda do animal

                    Se existirem compras de animais nos últimos (p) intervalos de tempo:
                        Marca a matriz (A) com o mesmo item do período anterior
                        Decide se irá comprar, vender ou manter animais
                        Decisão de compra:
                            Selecione entre 0 e (t) a quantidade de lotes de animais a serem comprados
                            Marca em (C) a quantidade de animais comprados
                        Decisão de venda, apenas se (a3) > 0:
                            Seleciona entre 0 e (a3) a quantidade de lotes de animais vendidos
                            Marca em (V) a quantidade de animais vendidos
                        Decisão de manter:
                            Não atualiza as matrizes (C) e (V)
                    
                    Caso contrário, todos os animais podem ser vendidos:
                        Decide se irá manter os animais ou encerrar o lote vendendo tudo e liberando para outra atividade
                        Se decidir por vender tudo:
                            Marca a matriz (V) com todos os animais do estoque (t)
                            Altera o enum para "ação inicial" e aplica a sequência de ações dela
        '''

        # Estrutura a forma de escolher entre fazer nada, plantar ou animais
        def action_decision(prob: float) -> int:
            if prob < 0.333:
                return 0
            elif prob < 0.666:
                crop = random.randint(0, self.crops - 1)
                return int(f"1{crop}")
            else:
                animal = random.randint(0, self.animals - 1)
                return int(f"2{animal}")
            
        # Verifica se existe um cultivo em crescimento
        def crop_in_growth(batch_times: List[int], current_time: int) -> bool:
            if batch_times[current_time - 1] != 0\
                and str(batch_times[current_time - 1])[0] == "1":
                # Faz o retorno até encontrar o tempo inicial
                element = batch_times[current_time - 1]
                count = 0
                for i in range(current_time - 1, -1, -1):
                    if batch_times[i] == element:
                        count = count + 1
                    else:
                        break

                # Verifica se ainda é preciso de tempo para crescer
                crop_position = int(str(element)[1:])
                if count % self.crop_time[crop_position] == 0:
                    return False
                else:
                    return True

            else:
                return False

        # Inicializa a matriz de controle de uso de terra
        decision_batch: List[List[int]] = []
        for l in range(0, self.batchs):
            batch_in_time: List[int] = []

            for i in range(0, self.times):
                if i == 0:
                    # Estamos no primeiro momento, decide o que fazer
                    action = action_decision(random.random())

                else:
                    # Verifica se existe um cultivo em crescimento
                    # Se existir, mantêm a ação atual, se não, elege outra
                    in_growth = crop_in_growth(batch_in_time, i)
                    if in_growth:
                        action = batch_in_time[i - 1]
                    else:
                        action = action_decision(random.random())

                batch_in_time.append(action)
            decision_batch.append(batch_in_time)

        # Cria a instância de variável de decisão de alocação de terras
        dv = DecisionVariable(shape=[self.batchs, self.times], default_value=0)
        dv.values = decision_batch
        self.dv_batch_use = dv

        # Para cada intervalo de tempo, define quantos animais serão comprados
        purchase_animals: List[List[int]] = []
        for j in range(0, self.animals):
            max_animals = int(self.batch_area / self.animal_area[j])

            for i in range(0, self.times):
                purchase_number = random.randint(0, max_animals)

                # Verifica se no mês atual existem lotes para a compra

                # Verifica o histórico de compras do animal e se existe espaço

        # Para cada intervalo de tempo, define quantos animais serão vendidos
        sell_animals: List[List[int]] = []

    def validate(self):
        pass


solution = Solution(
    times=13,
    batchs=10,
    crops=6,
    animals=5,
    crop_time=[1, 2, 3, 2, 4, 6],
    animal_area=[1, 2, 5, 4, 3],
    batch_area=30,
    animal_growth_time=[4, 4, 2, 2, 1],
    animal_manure=[0.4, 0.6, 0.2, 0.5, 0.05]
)
solution.random()



In [5]:
random.random()

0.9339040818852405